In [3]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as pil_img
import multiprocessing
from multiprocessing import Pool
import imutils
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
import random

In [ ]:
plt_params = {'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(plt_params)

In [18]:
class Image():
    def __init__(self, filename):
        self.filename = filename
        
    def open_image(self, open_dir):
        
        self.image_og = cv2.imread(open_dir+self.filename, cv2.IMREAD_UNCHANGED)
        if self.image_og.shape[2] == 4 or  self.image_og.shape[2] == 3:
            self.image_og =cv2.cvtColor(self.image_og, cv2.COLOR_BGR2RGB)
        self.height, self.width, channel = self.image_og.shape
    
    def resize_padding(self, desired_size):
        old_size = [self.width, self.height]
        ratio = float(desired_size)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        resized = cv2.resize(self.image_og,new_size)

        delta_w = desired_size - new_size[0]
        delta_h = desired_size - new_size[1]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)

        color = [0, 0, 0]
        self.saveimg = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    
    def resize_stretch(self, desired_size):
        self.saveimg = cv2.resize(self.image_og, (desired_size, desired_size), interpolation = cv2.INTER_AREA)

    def find_contours(self):
        self.gray = cv2.cvtColor(self.image_og, cv2.COLOR_BGR2GRAY)
        self.thresh = cv2.threshold(self.gray, 50, 255, cv2.THRESH_BINARY_INV)[1]
        
        self.contours, hierarchy = cv2.findContours(self.thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        self.contours = sorted(self.contours, key=cv2.contourArea, reverse = True)
        #plt.imshow(self.thresh)
        #plt.show()
        
    def morph_contours(self):
        kernel = np.ones((5,5), dtype='uint8')
        image_close = cv2.morphologyEx(self.thresh, cv2.MORPH_CLOSE, kernel)
        
        self.contours, hierarchy = cv2.findContours(image_close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
        draw=cv2.drawContours(self.thresh, self.contours, -1, (0,0,255), 2)
        draw = cv2.fillPoly(self.thresh, self.contours, color=(255,255,255))
        #plt.imshow(draw)
        #plt.show()

        self.contours, hierarchy = cv2.findContours(draw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #self.contours = sorted(contours, key=cv2.contourArea, reverse = True)

    def largest_contour(self):
        return sorted(self.contours, key=cv2.contourArea, reverse = True)[0] 
    
    def mask_background(self):
        '''Keeps the background all white surrounding the largest conotour.
        Places the largest contour on an array of all the same color'''
        print(self.image_og.shape[:2])
        mask = np.zeros(self.image_og.shape[:2], dtype="uint8")
        draw = cv2.drawContours(mask, [self.largest_contour()], 0, (255,255,255), -1)
        self.im = cv2.bitwise_and(self.im, self.im, mask=mask)
    
    def edges(self):
        min_threshold = 0.66 * np.mean(self.image_og)
        max_threshold = 1.33 * np.mean(self.image_og)
        edges = cv2.Canny(self.saveimg, min_threshold, max_threshold)
        return edges
        
    def contrast(self):
        return self.image_og.std()
        
    def laplacian(self):
        self.laplacian = cv2.Laplacian(self.gray,cv2.CV_64F).var()
        
    def random_color(self):
        rgbl=[255,0,0]
        random.shuffle(rgbl)
        return tuple(rgbl)

    def cutoff(self):
        #checking the percentage of the contour that touches the edge/border

        locations = np.where(self.thresh != 0)
        count = 0 #pixels touching border
        for xl,yl in zip(locations[0], locations[1]):
            if xl == 0 or yl == 0 or xl == self.height-1 or yl == self.width-1:
                count+=1
        cutoff_perc = (count/(2*self.height+2*self.width))*100
        return cutoff_perc

    def phi(self):
        rect = cv2.minAreaRect(self.largest_contour()) #box ONLY around the largest contour 
        #get length and width of contour
        x = rect[1][0]
        y = rect[1][1]      
        self.rect_length = max(x,y)
        self.rect_width = min(x,y)
        if self.rect_length != 0:
            return self.rect_width/self.rect_length
        else: 
            return 0
        
    def largest_contour_area(self):
        return cv2.contourArea(self.largest_contour())  
        
    def area(self):
        area = 0
        for c in self.contours:
            area+=cv2.contourArea(c)  
            return area
    
    def perim(self):
        if len(self.contours) == 1:
            return cv2.arcLength(self.largest_contour(), False)
        else: 
            return 0
    
    def hull_area(self):
        hull = cv2.convexHull(self.largest_contour())      
        return cv2.contourArea(hull)
        
    def perim_area(self):
        return self.perim()/self.area()
        
    def convex_perim(self, closed_cnt):
        hull = cv2.convexHull(self.largest_contour())
        return cv2.arcLength(hull, closed_cnt)
    
    def equiv_d(self):
        return np.sqrt(4*self.area()/np.pi)
    
    def circularity(self):
        perim = self.perim()
        hull_area = self.hull_area()
        if perim != 0 and hull_area != 0 and self.area() != 0 and len(self.contours) == 1:
            return (4.*np.pi*self.area())/(self.convex_perim(True)**2)
            #return (4*np.pi*self.area)/(perim**2)
        else:
            return 0
        
    def solidity(self):
        if self.hull_area() != 0. and len(self.contours) == 1:
            return float(self.area())/self.hull_area()
        else:
            return 0
    
    def complexity(self):
        if self.perim() != 0 and self.hull_area != 0 and self.area() != 0 and len(self.contours) == 1:
            return 10*(0.1-(self.area()/(np.sqrt(self.area()/self.hull_area())*self.perim()**2)))
        else:
            return 0
        
    def flip_imgs(self, save_dir):
        plt.imsave(save_dir+self.filename,np.array(self.saveimg))
        plt.imsave(save_dir+self.filename[:-4]+'_ud.png',np.flipud(self.saveimg))
        plt.imsave(save_dir+self.filename[:-4]+'_lr.png',np.fliplr(self.saveimg))
        plt.imsave(save_dir+self.filename[:-4]+'_ud_lr.png',np.flipud(np.fliplr(self.saveimg)))
        
    def rotate(self):
        # loop over the rotation angles
        for angle in np.arange(0, 360, 100):
            self.saveimg = imutils.rotate_bound(self.saveimg, angle)
            plt.imshow(self.saveimg)
            plt.show()
            
    def show_image(self):
        plt.imshow(self.image_og)
        plt.show()
        
    def show_resized(self):
        plt.imshow(self.saveimg)
        plt.show()

    def save_image(self, save_dir, flip = False):
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        if flip:
            self.flip_imgs(save_dir)
        else:
            #save single image, no flipping:
            cv2.imwrite(os.path.join(save_dir,str(self.filename)), np.array(self.saveimg))
#         plt.imshow(self.saveimg)
#         plt.show()
        
            

In [1]:
def main():

    campaign = 'MPACE'
    open_dirs = ['cpi_data/campaigns/'+campaign+'/single_imgs/', 'cpi_data/campaigns/'+campaign+'/bad_train/']
    save_dir = 'cpi_data/hand_labeled_pristine/'
    desired_size = 1000
    category = 'none'
    
    #Independent Variable
    good_bad = []
    
    #Dependent Variables
    lapl = []
    contours = []
    edges = []
    std = []
    height = []
    width = []
    cnt_area = []
    contrast = []
    circularity = []
    contrast = [] 
    solidity = []
    complexity = []
    equiv_d = []
    convex_perim = []
    perim_area = []
    hull_area = []
    perim = []
    phi = []
    cutoff = []

    for direct in open_dirs:
        
        for filename in os.listdir(direct):
            #want a good/bad index for every file
            if direct == open_dirs[0]:
                good_bad.append(0)
            else:
                good_bad.append(1)

            image = Image(filename)
            image.open_image(direct)
            image.find_contours()
            image.resize_stretch(desired_size)
            #image.save_image(save_dir, flip=True)
            if len(image.contours) > 1:
                image.morph_contours()
                
            image.laplacian()
            count_edge_px = np.count_nonzero(image.edges())
            
            lapl.append(image.laplacian)
            contours.append(len(image.contours))
            edges.append(count_edge_px)
            contrast.append(image.contrast())
            if count_edge_px > 0:
                std.append(np.std(np.nonzero(image.edges())))
            else:
                std.append(0)
            height.append(image.height)
            width.append(image.width)

            if len(image.contours)!=0 and image.area() > 0.00:
                cnt_area.append(cv2.contourArea(image.largest_contour()))
                solidity.append(image.solidity())
                complexity.append(image.complexity())
                equiv_d.append(image.equiv_d())
                convex_perim.append(image.convex_perim(True))
                perim_area.append(image.perim_area())
                hull_area.append(image.hull_area())
                perim.append(image.perim())
                phi.append(image.phi())
                circularity.append(image.circularity())
                cutoff.append(image.cutoff())

            else:
                cnt_area.append(0)
                solidity.append(0)
                complexity.append(0)
                equiv_d.append(0)
                convex_perim.append(0)
                perim_area.append(0)
                hull_area.append(0)
                perim.append(0)
                phi.append(0)
                circularity.append(0)
                cutoff.append(0)
                
            ####FOR CATEGORIZATION AFTER GOOD IMAGES RETURNED###
            if category == 'blank' and len(image.contours) == 0:
                image.resize_stretch(desired_size)
                image.show_resized()
                #image.save_image(save_dir, flip=True)

            if len(image.contours)!=0:
                #cutoff_perc = image.cutoff()

                laplacian = cv2.Laplacian(image.gray,cv2.CV_64F).var()

                if category == 'spheres':
                    if image.circularity() > 0.8 and image.solidity() > 0.8: 
                        #image.show_image()
                        save_dir = 'cpi_data/campaigns/MPACE/spheres/'
                        #image.save_image(save_dir, flip=False)
                    else:
                        #print(image.circularity(), image.solidity())
                        save_dir = 'cpi_data/campaigns/MPACE/no_spheres/'
                        #image.show_image()
                        image.save_image(save_dir, flip=False)

    #             if check_junk:
    #                 #include blurry, blank, and fragments
    #                 if laplacian > 1000 or len(image.contours) == 0 or \
    #                     (image.perim() !=-999 and image.area() != 0 and \
    #                     image.phi() < 0.5 and image.phi() !=-999 and \
    #                     cutoff_perc < 1 and 300 < laplacian < 700 and \
    #                     -999 < image.circularity() < 0.8 and \
    #                     (image.height < 60 and image.width < 60)):
    #                     #print(laplacian)
    #                     image.resize_stretch(desired_size)
    #                     #image.show_resized()
    #                 #if laplacian < 1000:


                if category == 'blurry':
                    if laplacian > 5000:    
                        image.resize_stretch(desired_size)
                        #image.save_image(save_dir, flip=True)
                        #image.show_resized()

                if category== 'fragment':
                    if image.perim() !=-999 and image.area() != 0 and \
                        image.phi() < 0.5 and image.phi() !=-999 and \
                        cutoff_perc < 1 and 300 < laplacian < 700 and \
                        -999 < image.circularity() < 0.8 and \
                        (image.height < 60 and image.width < 60):  
                        image.resize_stretch(desired_size) 
                        image.show_resized()

                if category == 'columns':
                    #print(image.phi())
                    #if len gt 1.9*wid and perim/area < 1.3*(2*(len+wid)/(len*wid))
                    if image.perim() !=-999 and image.area() != 0 and \
                        image.phi() < 0.4 and image.phi() !=-999 and \
                        cutoff_perc < 2. and 300 < laplacian < 700 and \
                        image.solidity() > 0.9 and image.rect_length > 40 and image.rect_width > 40:  
                        image.resize_stretch(desired_size) 
                        #image.save_image(save_dir, flip=True)

                if category == 'junk':
                    image.phi()
                    image.resize_stretch(desired_size)
                    if (image.length < 20 or image.width < 20) and image.phi() > 0.3 and \
                        image.phi() < 0.8 and \
                        cutoff_perc < 20 and laplacian < 1000 and laplacian > 600 and \
                        image.area() > 20.0 and image.circularity() < 0.7:
                        image.show_resized()

                if category == 'rimed_columns':
                    if image.perim() != -999 and image.solidity() > 0.3 and image.phi() < 0.5 and \
                        laplacian < 700 and image.perim_area() < .1:
                        image.resize_stretch(desired_size)
                        print(image.solidity(), image.complexity(), image.phi(), image.perim(), image.perim_area())
                        image.show_resized()

    #set up dictionary for good and bad data metrics
    dicts = {}
    keys = ['good_bad', 'lapl', 'contours', 'edges', 'std', 'height', 'width', 'cnt_area', \
           'contrast', 'circularity', 'solidity','complexity','equiv_d','convex_perim',\
           'perim_area', 'hull_area', 'perim', 'phi', 'cutoff']
    values =  [good_bad, lapl, contours, edges, std, height, width, cnt_area, \
           contrast, circularity, solidity, complexity, equiv_d, convex_perim,\
           perim_area, hull_area, perim, phi, cutoff]
    for key, val in zip(keys, values):
        dicts[key] = val
    #dicts = {'good_bad': good_bad, 'lapl': lapl, 'contours':contours, 'edges':edges, 'std':std, 'height':height, \
    #        'width':width, 'cnt_area':cnt_area, }
    
    df = pd.DataFrame(dicts)
    return df
            

In [ ]:
if __name__ == '__main__':
    %time df = main()

In [ ]:
df

In [ ]:
count = 0
for row in df['width']:
    if row == 1000:
        count+=1 
count

In [ ]:
df_new = df.drop(df[(df['height'] == 1000) & (df['width'] == 1000)].index)

In [ ]:
df.isnull().sum()

# Regression

In [ ]:
predictors = ['lapl', 'contours', 'edges', 'std', 'height', 'width', 'cnt_area', \
           'contrast', 'circularity', 'solidity','complexity','equiv_d','convex_perim',\
           'perim_area','hull_area','perim','phi', 'cutoff']
# predictors = ['lapl', 'contours', 'edges', 'std', 'width', 'cnt_area', 'contrast', 'hull_area', 'solidity', 'phi',
#              'perim_area', 'cutoff', 'circularity', 'convex_perim', 'complexity']

#X = df.drop('good_bad', 1)
X = df_new[predictors]
y = df_new['good_bad']

logit_model=sm.Logit(y, X, missing='drop')
result=logit_model.fit()
print(result.summary2())

In [ ]:
f = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=14, rotation=45)
plt.yticks(range(df.shape[1]), df.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
sns.regplot(x=df['edges'], y=df['good_bad'], y_jitter=0.03, logistic = True)

In [ ]:
sns.regplot(x=df['width'], y=df['good_bad'], y_jitter=0.03, logistic = True)

In [ ]:
sns.regplot(x=df['cnt_area'], y=df['good_bad'], y_jitter=0.03, logistic = True)

In [ ]:
sns.regplot(x=df['solidity'], y=df['good_bad'], y_jitter=0.03, logistic = True)

In [ ]:
sns.regplot(x=df['contrast'], y=df['good_bad'], y_jitter=0.03, logistic = True)

In [ ]:
sns.regplot(x=df['perim'], y=df['good_bad'], y_jitter=0.03, logistic = True)

# PCA

In [ ]:
X_pca = df_new[df_new['contours'] == 1].drop('good_bad', 1)
y_pca = df_new[df_new['contours'] == 1]['good_bad']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_pca, test_size=0.2, random_state=0)

In [ ]:
pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
explained_variance = pca.explained_variance_ratio_
explained_variance

In [ ]:
plt.plot(np.arange(len(explained_variance)), explained_variance, marker='o')
plt.xlabel("Principal Component")
plt.ylabel('Variance')

In [ ]:
fig = plt.figure(1, figsize=(7, 7))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=90, azim=10)
ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: %.2f' %(accuracy_score(y_test, y_pred)*100))

# Support Vector Classification

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print('Accuracy: %.2f' %(accuracy_score(y_test, y_pred)*100))

# make new prediction based on logit model

In [ ]:
campaign = '2004_Midcix'
open_dir = 'cpi_data/campaigns/'+campaign+'/single_imgs/'
save_dir_good = 'cpi_data/campaigns/'+campaign+'/good/'
save_dir_bad = 'cpi_data/campaigns/'+campaign+'/bad/'
desired_size = 1000

bad = 0
good = 0
for filename in os.listdir(open_dir):
    image = Image(filename)
    image.open_image(open_dir)
    image.find_contours()
    image.resize_stretch(desired_size)
    image.laplacian()
    if len(image.contours) > 1:
        image.morph_contours()
    
    lapl = image.laplacian
    contours = len(image.contours)
    edges = np.count_nonzero(image.edges())       
    if edges > 0:
        std=np.std(np.nonzero(image.edges()))
    else:
        std=0
    height = image.height
    width = image.width
    contrast = image.contrast()

    if len(image.contours)!= 0 and image.area() > 0.00:
        
        cnt_area = cv2.contourArea(image.largest_contour())
        solidity=image.solidity()
        complexity=image.complexity()
        equiv_d=image.equiv_d()
        convex_perim=image.convex_perim(True)
        perim_area=image.perim_area()
        hull_area=image.hull_area()
        perim=image.perim()
        phi=image.phi()
        circularity=image.circularity()
        cutoff = image.cutoff()
    else:
        cnt_area=0
        solidity=0
        complexity=0
        equiv_d=0
        convex_perim=0
        perim_area=0
        hull_area=0
        perim=0
        phi=0
        circularity=0
        cutoff = 0

    dicts = {}
    keys = ['lapl', 'contours', 'edges', 'std', 'height', 'width', 'cnt_area', \
           'contrast', 'circularity', 'solidity','complexity','equiv_d','convex_perim',\
           'perim_area', 'hull_area', 'perim', 'phi', 'cutoff']
    values =  [lapl, contours, edges, std, height, width, cnt_area, \
           contrast, circularity, solidity, complexity, equiv_d, convex_perim,\
           perim_area, hull_area, perim, phi, cutoff]

    for key, val in zip(keys, values):
        dicts[key] = val
    df = pd.DataFrame(dicts, index=[0])
    
    #Regression model prediction
    pred = result.predict(df)
    
    df = pca.transform(df)
    #SVC prediction
    #pred = clf.predict(df)
    
    #PCA prediction
    #pred = classifier.predict(df)
    
    #print(pred[0])
    if pred[0] < 0.5:
        if len(image.contours) != 0:
            print(image.cutoff())
            print("GOOD")
            image.show_image()
            plt.show()
            #image.save_image(save_dir_good)
            good += 1
    else:
#         print("BAD")
#         image.show_image()
#         plt.show()
         #image.save_image(save_dir_bad)
        bad += 1

print((good/(good+bad))*100)

In [ ]:
def reject_outliers(data):
    m = 4
    u = np.nanmean(data)
    s = np.std(data)
    filtered = [e for e in data if (u - m * s < e < u + m * s)]
    return filtered

std_filt = reject_outliers(std)
contours_filt=reject_outliers(contours)
edges_filt = reject_outliers(edges)
lapl_arr_filt = reject_outliers(lapl_arr)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1) 
ax.hist(lapl_arr_filt, bins = 10, density=True )
fig, ax = plt.subplots(nrows=1, ncols=1) 
ax.hist(std_filt)
fig, ax = plt.subplots(nrows=1, ncols=1) 
ax.hist(contours_filt, bins = 70)
fig, ax = plt.subplots(nrows=1, ncols=1) 
ax.hist(edges_filt, bins = 70, density=True)
ax.set_xlim(0,1000)

In [16]:
def main(filename):  
    save_dir = '../../cpi_data/OLYMPEX/new_labels3/aggs/'
    desired_size = 1000
    image = Image(filename)
    
    image.open_image(open_dir)
    image.mask_background()
    #image.find_contours()
    #laplacian = cv2.Laplacian(image.gray,cv2.CV_64F).var()
  
    #if len(image.contours)!=0 and laplacian > 4000:

    #image.resize(desired_size) 
    #image.save_image(save_dir)


In [23]:
%%time
if __name__ == '__main__':
    open_dir = '../cpi_data/campaigns/OLYMPEX/single_imgs1/'
    filenames = os.listdir(open_dir)
    main(filenames[40])
    #print(multiprocessing.cpu_count())  #80
    #with Pool(multiprocessing.cpu_count()) as p:
    #    p.map(main, filenames)
    

(1, 1)


AttributeError: 'Image' object has no attribute 'contours'

spheres blurriness values

In [ ]:
lapl_arr = np.array(lapl_arr)
lapl_arr[lapl_arr < 2.] = np.nan

In [ ]:
#second mode starts at 175
fig, ax = plt.subplots( nrows=1, ncols=1 ) 
ax.hist(lapl_arr, bins = 70)
#ax.set_xlim(100,300)
#fig.savefig('sphere_lapl_hist.png')

In [ ]:
#read csv's
import csv

def readFile(filename):
    files = []
    areas = []
    perims = []
    lengths = []
    widths = []
    Cs = []
    solidities = []
    cplxs = []
    with open(filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        next(csvReader, None)  # skip the headers
        for row in csvReader:
            
            files.append(row[0])
            areas.append(float(row[1])) 
            perims.append(float(row[2])) 
            lengths.append(float(row[3]))
            widths.append(float(row[4]))
            Cs.append(float(row[5]))
            solidities.append(float(row[6]))
            cplxs.append(float(row[7]))

    return files, areas, perims, lengths, widths, Cs, solidities, cplxs


files, areas, perims, lengths, widths, Cs, solidities, cplxs = readFile('pristine_columns_out_file.csv')
